In [33]:
import numpy as np

In [34]:
#Parameters
tf=120
sequence = 30
TTP = 0.8 # Train Test Percentage 
K = 5
epoch = 100
early = 10
lstm1 = 64
lstm2 = 64
random = 0 # on=1 off=0
KFold = 1 # on=1 off=0

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import os

directory = r"/content/drive/MyDrive/Paper3/Data"

# get a list of all .txt files in the directory
txt_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.txt')]

# print the number of .txt files found
print("Number of txt files:", len(txt_files))

# read the contents of each .txt file and store them in an array
Data = np.zeros((len(txt_files),50000,4))
for file_path in txt_files:
    for i in range(len(txt_files)):
        data = np.loadtxt(txt_files[i], dtype=object)
        data = [line.split(',') for line in data]

        # Convert the resulting list of lists to a NumPy array with the appropriate data types
        data = np.array(data, dtype=np.float32)
        #data = data.astype(np.int32)
        A = np.zeros((50000-data.shape[0], data.shape[1]))
        Data[i] = np.concatenate((data, A))

# # print the contents of the array
# print("File contents:", file_contents)

Number of txt files: 11


In [37]:
if random ==1:
    Frames = np.zeros((frames.shape[0],frames.shape[1],2),dtype='float') 
    A = np.array(range(12))
    random.shuffle(A)
    for i in range(12):
        Frames[i] = frames[A[i]]
    frames = Frames
    
    
if KFold == 1:
    
    Size = round(len(txt_files)*(1-TTP))
    print("Size: ",Size)
    data_zeros = np.zeros((Data.shape[0], Data.shape[1], Data.shape[2]),dtype='float') 
    startpoint = (K-1)*Size
    A = np.array(range(len(txt_files)))
    for i in range(Size):
        data_zeros[i] = Data[startpoint]
        A = np.delete(A, Size*(K-1))
        startpoint = startpoint + 1
    for i in range(len(txt_files)-Size):
        data_zeros[i+Size] = Data[A[i]]
    Data = data_zeros

Size:  2


In [38]:
import pandas as pd

# Load the Excel file into a DataFrame
df = pd.read_excel(r'/content/drive/MyDrive/Paper3/Properties.xlsx', sheet_name='Sheet1')

# Select the values from row 3 and onwards
Properties = df.values[1:]

In [39]:
import numpy as np

# Assume the data array is already loaded into a variable named 'data'

# Find the indices where the values are "N" or "F"
idx_n = np.where(Properties == np.array(["N"], dtype=str))
idx_f = np.where(Properties == np.array(["F"], dtype=str))

# Replace "N" with "1"
Properties[idx_n] = 1

# Replace "F" with "2"
Properties[idx_f] = 2


In [40]:
Properties = np.array(Properties)
Properties = Properties.astype(float)

In [41]:
resulotion = 0.04
Dim = round((Properties[-1,0]-Properties[0,0])/resulotion)
Properties_res = np.zeros((Dim, Properties.shape[1]))
Data_res = np.zeros((len(txt_files), Dim, Data.shape[2]))
t0 = 5
tf = 620.8
t = t0
Data_inter = np.zeros(len(txt_files),dtype="int")
Properties_inter = 0
#for j in range(len(txt_files)):
for j in range(1):
    for i in range(Data[j].shape[0]):
        if Data[j,i,0] >= t0:
            Data_inter[j] = i - 1
            break
for i in range(Dim):
    Properties_res[i] = Properties[Properties_inter]
    Properties_res[i,0] = t
    if t >= Properties[Properties_inter+1,0]:
        Properties_inter += 1
    for j in range(len(txt_files)):
        if Data[j,Data_inter[j]+2,0]==0:
            continue
        Data_res[j,i] = Data[j, Data_inter[j]]
        Data_res[j,i,0] = t
        while True:
            if t >= Data[j, Data_inter[j] + 1,0]:
                Data_inter[j] += 1
            else:
                break
    
    t += resulotion

In [42]:
# Number 1
XBminN1 = 120 #X Body min Near
XBmaxN1 = 160 #X Body max Near

XBminF1 = 150 #X Body min Far
XBmaxF1 = 170 #X Body max Far


# Number 2
XBminN2 = 160 #X Body min Near
XBmaxN2 = 195 #X Body max Near

XBminF2 = 170 #X Body min Far
XBmaxF2 = 190 #X Body max Far


# Number 3
XBminN3 = 195 #X Body min Near
XBmaxN3 = 230 #X Body max Near

XBminF3 = 190 #X Body min Far
XBmaxF3 = 220 #X Body max Far


In [43]:
Model_data = np.zeros((Data_res.shape[0]*Data_res.shape[1],Properties_res.shape[1]))
Model_label = np.zeros((Data_res.shape[0]*Data_res.shape[1]), dtype="O")
inter = 0
for z in range(Data_res.shape[0]):
    for j in range(Data_res.shape[1]):
        if Data_res[z,j,0] == 0:
            continue
        Model_data[inter] = Properties_res[j]
        Model_label[inter] = "No Data"

        P = 0 # number of person 1
        if Properties_res[j,6*P+1] == 1: #if the person is present
            if Properties_res[j,6*P+2] == 1: #if the person is Near or Far
                if Properties_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminN1<Data_res[z,j,2]<XBmaxN1: #Body
                        Model_label[inter] = "Number 1"
                            
                elif Properties_res[j,6*P+3] == 7:  #if the person is stationary or moving
                    if 100<Data_res[z,j,2]<XBmaxN1: #Body
                        Model_label[inter] = "Number 1"

                        
            elif Properties_res[j,6*P+2] == 2: #if the person is Near or Far
                if Properties_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminF1<Data_res[z,j,2]<XBmaxF1: #Body
                        Model_label[inter] = "Number 1"

                elif Properties_res[j,6*P+3] == 7:  #if the person is stationary or moving
                    if 100<Data_res[z,j,2]<XBmaxN1: #Body
                        Model_label[inter] = "Number 1"


        P = 1 # number of person 2
        if Properties_res[j,6*P+1] == 1: #if the person is present
            if Properties_res[j,6*P+2] == 1: #if the person is Near or Far
                if Properties_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminN2<Data_res[z,j,2]<XBmaxN2: #Body
                        Model_label[inter] = "Number 2"
                            
                elif Properties_res[j,6*P+3] == 7:  #if the person is stationary or moving
                    if Properties_res[j,6*P+6] <=0:  #if the person is leaving or entering from left side
                        if 100<Data_res[z,j,2]<180: #Body
                            Model_label[inter] = "Number 2"
                    elif Properties_res[j,6*P+6] >=0:  #if the person is leaving or entering from left side
                        if 180<Data_res[z,j,2]<250: #Body
                            Model_label[inter] = "Number 2"
                            

            elif Properties_res[j,6*P+2] == 2: #if the person is Near or Far
                if Properties_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminF2<Data_res[z,j,2]<XBmaxF2: #Body
                        Model_label[inter] = "Number 2"

                            
                elif Properties_res[j,6*P+3] == 7:  #if the person is stationary or moving
                    if Properties_res[j,6*P+6] <=0:  #if the person is leaving or entering from left side
                        if 100<Data_res[z,j,2]<180: #Body
                            Model_label[inter] = "Number 2"
                    elif Properties_res[j,6*P+6] >=0:  #if the person is leaving or entering from left side
                        if 180<Data_res[z,j,2]<250: #Body
                            Model_label[inter] = "Number 2"
                            

        P = 2 # number of person 3
        if Properties_res[j,6*P+1] == 1: #if the person is present
            if Properties_res[j,6*P+2] == 1: #if the person is Near or Far
                if Properties_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminN3<Data_res[z,j,2]<XBmaxN3: #Body
                        Model_label[inter] = "Number 3"
                            
                elif Properties_res[j,6*P+3] == 7:  #if the person is stationary or moving
                    if XBminN3<Data_res[z,j,2]<250: #Body
                        Model_label[inter] = "Number 3"

            elif Properties_res[j,6*P+2] == 2: #if the person is Near or Far
                if Properties_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminF3<Data_res[z,j,2]<XBmaxF3: #Body
                        Model_label[inter] = "Number 3"
                        
                elif Properties_res[j,6*P+3] == 7:  #if the person is stationary or moving
                        if XBminF3<Data_res[z,j,2]<250: #Body
                            Model_label[inter] = "Number 3"
        inter +=1

In [44]:
Model_data = np.array(Model_data[0:inter])
Model_label = np.array(Model_label[0:inter])

In [45]:
Model_data.shape

(164061, 19)

In [46]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler = scaler.fit(Model_data)
Model_data_normal = scaler.transform(Model_data)  #scale the data

Model_data_normal = np.delete(Model_data_normal, 0, axis=1)

In [47]:
def Labell(String):
    if String == "Number 1":
        return 0
    elif String == "Number 2":
        return 1
    elif String == "Number 3":
        return 2

In [48]:
# Model_data_normal_sequence = np.zeros((Model_data_normal.shape[0]-sequence+1,sequence,Model_data_normal.shape[1]),dtype='float')
# for i in range(Model_data_normal_sequence.shape[0]):
#     for j in range(sequence):
#         Model_data_normal_sequence[i,j] = Model_data_normal[i+j]
        
# Model_data_normal_sequence = np.delete(Model_data_normal_sequence, -1, axis = 0)

In [49]:
Model_data_normal_sequence = np.zeros((Model_data_normal.shape[0],sequence,Model_data_normal.shape[1]),dtype='float')
Model_label_sequence = np.zeros((Model_data_normal.shape[0],1),dtype='int')
inter = 0
Persons_inter = np.zeros(len(txt_files))
inter2 = 0
for i in range(Model_data_normal_sequence.shape[0]-sequence):
    if Model_data[i, 0] == 5:
        Persons_inter[inter2] = inter
        inter2 += 1
    if Model_label[i+sequence] != "No Data" and (Model_data[i+sequence, 0] - Model_data[i, 0]) > 0:
        Model_label_sequence[inter] = Labell(Model_label[i+sequence])
        for j in range(sequence):
            Model_data_normal_sequence[inter,j] = Model_data_normal[i+j]
        inter += 1
        
Model_data_normal_sequence = np.array(Model_data_normal_sequence[0:inter])
Model_label_sequence = np.array(Model_label_sequence[0:inter])

In [50]:
Persons_inter

array([     0.,  14980.,  23535.,  38604.,  51505.,  66447.,  80448.,
        94827., 109418., 122581., 135243.])

In [51]:
Model_data_normal_sequence.shape

(150151, 30, 18)

In [52]:
train_data = np.array(Model_data_normal_sequence[int(Persons_inter[Size]):Model_data_normal_sequence.shape[0]])
train_label = np.array(Model_label_sequence[int(Persons_inter[Size]):Model_label_sequence.shape[0]])
test_data = np.array(Model_data_normal_sequence[0:int(Persons_inter[Size])])
test_label = np.array(Model_label_sequence[0:int(Persons_inter[Size])])

In [53]:
print("train_data:",train_data.shape)
print("train_label:",train_label.shape)
print("test_data:",test_data.shape)
print("test_label:",test_label.shape)

train_data: (126616, 30, 18)
train_label: (126616, 1)
test_data: (23535, 30, 18)
test_label: (23535, 1)


In [54]:
from tensorflow.keras.utils import to_categorical

#X_train = train_data.reshape(train_data.shape[0],sequence, -1)
one_hot_encoded_labels_train = to_categorical(train_label)

#X_test = test_data.reshape(test_data.shape[0],sequence, -1)
one_hot_encoded_labels_test = to_categorical(test_label)

In [55]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

model = Sequential()
model.add(LSTM(lstm1, activation='tanh',input_shape=(train_data.shape[1],train_data.shape[2]), return_sequences= True))
model.add(LSTM(lstm2, activation='tanh',input_shape=(train_data.shape[1],train_data.shape[2]), return_sequences= False))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(one_hot_encoded_labels_train[0].shape[0], activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 30, 64)            21248     
                                                                 
 lstm_3 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 flatten_1 (Flatten)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                                 
Total params: 54,467
Trainable params: 54,467
Non-trainable params: 0
_________________________________________________________________


In [56]:
early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = early, mode = 'max', restore_best_weights = True)
import keras
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import schedules

model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

model_training_history = model.fit(x = train_data, y = one_hot_encoded_labels_train, epochs = epoch , batch_size = 20,
                                                    shuffle = True, validation_data=(test_data, one_hot_encoded_labels_test),
                                                    callbacks = [early_stopping_callback])



Epoch 1/100
6331/6331 [==============================] - 55s 8ms/step - loss: 0.6347 - accuracy: 0.7018 - val_loss: 0.5638 - val_accuracy: 0.7519
Epoch 2/100
6331/6331 [==============================] - 50s 8ms/step - loss: 0.6056 - accuracy: 0.7172 - val_loss: 0.5443 - val_accuracy: 0.7745
Epoch 3/100
6331/6331 [==============================] - 49s 8ms/step - loss: 0.5976 - accuracy: 0.7215 - val_loss: 0.5311 - val_accuracy: 0.7745
Epoch 4/100
6331/6331 [==============================] - 48s 8ms/step - loss: 0.5905 - accuracy: 0.7241 - val_loss: 0.5450 - val_accuracy: 0.7584
Epoch 5/100
6331/6331 [==============================] - 48s 8ms/step - loss: 0.5840 - accuracy: 0.7274 - val_loss: 0.5555 - val_accuracy: 0.7714
Epoch 6/100
6331/6331 [==============================] - 49s 8ms/step - loss: 0.5826 - accuracy: 0.7282 - val_loss: 0.5365 - val_accuracy: 0.7812
Epoch 7/100
6331/6331 [==============================] - 49s 8ms/step - loss: 0.5793 - accuracy: 0.7292 - val_loss: 0.5457 -

In [57]:
y_pred_train = model.predict(train_data)
y_pred_test = model.predict(test_data)

736/736 [==============================] - 2s 3ms/step


In [58]:
def accuracymodel1(y_pred,train_label):
    SumCorrect = 0
    for i in range(y_pred.shape[0]):
        a = np.array(y_pred[i])
        a = a.argmax()
        if a==train_label[i,0]:
            SumCorrect = SumCorrect + 1
    return SumCorrect/y_pred.shape[0]

In [59]:
accuracymodel1(y_pred_train,train_label)
accuracymodel1(y_pred_test,test_label)

0.7836838750796685

In [60]:
def accuracymodel2(y_pred,train_label):
    SumCorrect = 0
    for i in range(y_pred.shape[0]):
        a = np.array(y_pred[i])
        b = np.array(y_pred[i])
        a = a.argmax()
        b[a] = 0
        b = b.argmax()
        
        if a==train_label[i,0] or b==train_label[i,0]:
            SumCorrect = SumCorrect + 1
    return SumCorrect/y_pred.shape[0]

In [61]:
accuracymodel2(y_pred_train,train_label)
accuracymodel2(y_pred_test,test_label)

0.9713618015721266

In [62]:
def accuracymodel3(y_pred,train_label):
    SumCorrect = 0
    for i in range(y_pred.shape[0]):
        a = np.array(y_pred[i])
        b = np.array(y_pred[i])
        c = np.array(y_pred[i])
        a = a.argmax()
        b[a] = 0
        b = b.argmax()
        c[a] = 0
        c[b] = 0
        c = c.argmax()
        
        if a==train_label[i,0] or b==train_label[i,0] or c==train_label[i,0]:
            SumCorrect = SumCorrect + 1
    return SumCorrect/y_pred.shape[0]

In [63]:
accuracymodel3(y_pred_train,train_label)
accuracymodel3(y_pred_test,test_label)

1.0

In [64]:
A = np.array([[accuracymodel1(y_pred_train,train_label), accuracymodel2(y_pred_train,train_label),accuracymodel3(y_pred_train,train_label)],
             [accuracymodel1(y_pred_test,test_label), accuracymodel2(y_pred_test,test_label),accuracymodel3(y_pred_test,test_label)]])
print(A)
Name = "LSTM-K"+str(K)+"-TTP"+str(TTP)+"-sequence" +str(sequence)
np.savetxt(r"/content/drive/MyDrive/Paper3/results"+"/"+Name+'.csv', A, delimiter=',')   # X is an array
model.save(r"/content/drive/MyDrive/Paper3/results"+"/"+Name+'.h5')  # creates a HDF5 file 'my_model.h5'

[[0.73448063 0.96293517 1.        ]
 [0.78368388 0.9713618  1.        ]]
